In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp2').getOrCreate()

In [2]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer

In [3]:
sentenceData = spark.createDataFrame([(0.0,'Hi I heard about Spark'),
                                     (1.0,'I wish I am a bad guy'),
                                     (2.0,'random forest is a good model')],
                                    ["label","sentence"])

In [4]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  1.0|I wish I am a bad...|
|  2.0|random forest is ...|
+-----+--------------------+



In [5]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [6]:
words_data = tokenizer.transform(sentenceData)

In [7]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  1.0|I wish I am a bad...|[i, wish, i, am, ...|
|  2.0|random forest is ...|[random, forest, ...|
+-----+--------------------+--------------------+



In [8]:
words_data.show(truncate= False)

+-----+-----------------------------+------------------------------------+
|label|sentence                     |words                               |
+-----+-----------------------------+------------------------------------+
|0.0  |Hi I heard about Spark       |[hi, i, heard, about, spark]        |
|1.0  |I wish I am a bad guy        |[i, wish, i, am, a, bad, guy]       |
|2.0  |random forest is a good model|[random, forest, is, a, good, model]|
+-----+-----------------------------+------------------------------------+



In [9]:
hashing_tf = HashingTF(inputCol='words',outputCol='rawFeatures')

In [10]:
featurized_data = hashing_tf.transform(words_data)

In [11]:
featurized_data.show()

+-----+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawFeatures|
+-----+--------------------+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[24417,49...|
|  1.0|I wish I am a bad...|[i, wish, i, am, ...|(262144,[20719,24...|
|  2.0|random forest is ...|[random, forest, ...|(262144,[15889,92...|
+-----+--------------------+--------------------+--------------------+



In [12]:
idf = IDF(inputCol='rawFeatures',outputCol='features')

In [13]:
idf_model = idf.fit(featurized_data)

In [14]:
rescaled_data = idf_model.transform(featurized_data)

In [15]:
rescaled_data.select(['label','features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262144,[24417,49...|
|  1.0|(262144,[20719,24...|
|  2.0|(262144,[15889,92...|
+-----+--------------------+



In [16]:
from pyspark.ml.feature import CountVectorizer

In [17]:
df = spark.createDataFrame([(0,"a b c".split(" ")),
                           (1, "a b b c a".split(" "))],
                           ["id","words"])

In [18]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [19]:
cv= CountVectorizer(inputCol="words",outputCol="features",vocabSize=3,minDF=2.0)

In [20]:
model = cv.fit(df)

In [21]:
result = model.transform(df)

In [22]:
result.show()

+---+---------------+--------------------+
| id|          words|            features|
+---+---------------+--------------------+
|  0|      [a, b, c]|(3,[0,1,2],[1.0,1...|
|  1|[a, b, b, c, a]|(3,[0,1,2],[2.0,2...|
+---+---------------+--------------------+

